# Atomic Agents × Opik – Context-Provider Demo



This tutorial demonstrates how to use `OpikContextProvider` to inject current trace

information into agent prompts for better observability and debugging.



**Key Concepts:**

* Context providers inject trace metadata into system prompts

* This allows agents to reference their execution context

* Useful for multi-agent workflows and debugging

* The trace context is automatically managed by Opik's tracking



**Simple Workflow:**

1. `MainAgent` – processes user requests and may call a specialist

2. `SpecialistAgent` – handles specific domain questions

3. Both agents can access the current trace ID through their context provider

## Installation

```bash

pip install "opik[atomic_agents]" atomic-agents instructor

```

In [ ]:
from typing import Optional

from pydantic import Field
from instructor.client import Instructor

from atomic_agents.agents.base_agent import (
    BaseAgent as BaseChatAgent,
    BaseAgentConfig as BaseChatAgentConfig,
)
from atomic_agents.lib.base.base_io_schema import BaseIOSchema

from opik.integrations.atomic_agents import track_atomic_agents, OpikContextProvider
from opik import context_storage

# One-liner enabling Opik tracing of BaseAgent instances
track_atomic_agents(project_name="context-provider-demo")


In [14]:
class UserQuery(BaseIOSchema):
    """User question schema"""
    message: str = Field(..., description="User question or request")


class AgentResponse(BaseIOSchema):
    """Agent response schema"""
    message: str = Field(..., description="Agent response")
    trace_context: Optional[str] = Field(None, description="Current trace context for debugging")



In [15]:
def demonstrate_context_provider():
    """Demonstrate context provider functionality with mock trace data"""
    
    print("=== OpikContextProvider Demonstration ===\n")
    
    # Example 1: Basic context provider
    print("1. Basic Context Provider:")
    basic_provider = OpikContextProvider(project_name="demo-project")
    print(f"   Without trace_id: '{basic_provider.get_info()}'")
    
    basic_provider.set_trace_id("trace-12345")
    print(f"   With trace_id: '{basic_provider.get_info()}'")
    
    # Example 2: Context provider with metadata
    print("\n2. Context Provider with Metadata:")
    advanced_provider = OpikContextProvider(
        project_name="demo-project",
        trace_id="trace-67890",
        include_metadata=True,
        metadata={
            "agent_role": "specialist",
            "workflow_type": "delegation",
            "user_id": "user-123"
        }
    )
    print(f"   Full context:\n{advanced_provider.get_info()}")
    
    # Example 3: What gets injected into prompts
    print("\n3. System Prompt Injection Example:")
    print("   When an agent uses this context provider, this information")
    print("   gets automatically injected into the system prompt:")
    print("   ```")
    print(f"   {advanced_provider.get_info()}")
    print("   ```")
    
    return basic_provider, advanced_provider



In [16]:
class MainAgent(BaseChatAgent):
    """Main agent that can delegate to specialists when needed"""
    input_schema = UserQuery
    output_schema = AgentResponse
    
    def get_response(self, **kwargs) -> AgentResponse:
        """Process user request and optionally delegate to specialist"""
        user_input = self.current_user_input
        
        # Get current trace context for debugging
        trace_data = context_storage.get_current_context_instance().get_trace_data()
        current_trace_id = trace_data.id if trace_data else "no-trace"
        
        if user_input and "specialist" in user_input.message.lower():
            # Delegate to specialist - this happens within the same trace context
            specialist_response = call_specialist_agent(user_input.message)
            return AgentResponse(
                message=f"Specialist says: {specialist_response}",
                trace_context=f"Main agent trace: {current_trace_id}"
            )
        
        return AgentResponse(
            message="I'm the main agent. Ask for a 'specialist' to see delegation!",
            trace_context=f"Main agent trace: {current_trace_id}"
        )



In [17]:
def call_specialist_agent(question: str) -> str:
    """Call specialist agent within the current trace context"""
    
    # Create context provider that will inject trace info into prompts
    # The trace_id will be automatically detected from current context
    trace_data = context_storage.get_current_context_instance().get_trace_data()
    
    context_provider = OpikContextProvider(
        project_name="context-provider-demo",
        trace_id=trace_data.id if trace_data else None,
        include_metadata=True,
        metadata={
            "agent_role": "specialist",
            "delegation_source": "main_agent"
        }
    )

    class SpecialistAgent(BaseChatAgent):
        """Specialist agent with context awareness"""
        input_schema = UserQuery
        output_schema = AgentResponse

        def get_response(self, **kwargs) -> AgentResponse:
            """Provide specialist response with context awareness"""
            # In a real scenario, the context provider would inject
            # trace information into the system prompt automatically
            specialist_answer = f"Specialist analysis complete! (Question: {question[:20]}...)"
            
            trace_data = context_storage.get_current_context_instance().get_trace_data()
            current_trace_id = trace_data.id if trace_data else "no-trace"
            
            return AgentResponse(
                message=specialist_answer,
                trace_context=f"Specialist trace: {current_trace_id}"
            )

    # Simple mock client for demonstration
    def mock_create(**kwargs):
        return AgentResponse(
            message="Mock specialist response",
            trace_context="mock-context"
        )

    dummy_client = Instructor(client=None, create=mock_create)

    agent = SpecialistAgent(
        config=BaseChatAgentConfig(model="gpt-3.5-turbo", client=dummy_client),
    )
    
    # Register the context provider - this injects trace info into prompts
    agent.register_context_provider("opik_context", context_provider)
    
    # This call happens within the existing trace context
    response = agent.run(UserQuery(message=question))
    return response.message



In [18]:
def demonstrate_shared_trace_context():
    """Demonstrate how context providers would work with shared trace context"""
    
    print("=== Shared Trace Context Demo ===\n")
    print("Note: In a real multi-agent workflow, you would use opik_context.trace_context()")
    print("to wrap the entire workflow. Here we simulate the concept:\n")
    
    # Simulate a shared trace ID that would be maintained across agents
    shared_trace_id = "019828bd-shared-trace-demo-1234"
    
    print(f"Simulated shared trace ID: {shared_trace_id}")
    
    # Create context providers that would use the same trace ID
    main_context_provider = OpikContextProvider(
        project_name="context-provider-demo",
        trace_id=shared_trace_id,
        include_metadata=True,
        metadata={"agent_role": "main", "workflow_step": "initial_processing"}
    )
    
    specialist_context_provider = OpikContextProvider(
        project_name="context-provider-demo",
        trace_id=shared_trace_id,  # Same trace ID!
        include_metadata=True,
        metadata={"agent_role": "specialist", "workflow_step": "detailed_analysis"}
    )
    
    print(f"\n1. Main agent context (shared trace {shared_trace_id[:8]}...):")
    print(f"   {main_context_provider.get_info()}")
    
    print(f"\n2. Specialist agent context (same trace {shared_trace_id[:8]}...):")
    print(f"   {specialist_context_provider.get_info()}")
    
    print(f"\n3. Both agents would inject context from the SAME trace:")
    print(f"   - Main agent sees: trace_id: {shared_trace_id}")
    print(f"   - Specialist agent sees: trace_id: {shared_trace_id}")
    print(f"   - This enables linking multi-agent conversations!")
    
    print(f"\n4. Real implementation pattern:")
    print(f"   ```python")
    print(f"   # Wrap entire workflow in shared trace")
    print(f"   with opik_context.trace_context(trace_data=shared_trace, client=client):")
    print(f"       # All agent.run() calls within this block share the trace")
    print(f"       main_response = main_agent.run(query)")
    print(f"       specialist_response = specialist_agent.run(follow_up)")
    print(f"   ```")


def main():
    # First demonstrate context provider basics
    basic_provider, advanced_provider = demonstrate_context_provider()
    
    print("\n" + "="*60)
    print("=== Individual Agent Runs (Default Behavior) ===\n")
    
    def mock_main_create(**kwargs):
        return AgentResponse(
            message="Mock main response",
            trace_context="mock-main-context"
        )

    dummy_main_client = Instructor(client=None, create=mock_main_create)

    # Create main agent with context provider
    main_context_provider = OpikContextProvider(
        project_name="context-provider-demo",
        include_metadata=True,
        metadata={"agent_role": "main", "workflow_type": "individual_demo"}
    )

    main_agent = MainAgent(
        config=BaseChatAgentConfig(model="gpt-3.5-turbo", client=dummy_main_client),
    )
    main_agent.register_context_provider("opik_context", main_context_provider)

    # Test 1: Regular request
    print("Test 1: Regular Request")
    response1 = main_agent.run(UserQuery(message="Hello, how are you?"))
    print(f"  Response: {response1.message}")
    print(f"  Context: {response1.trace_context}")

    # Capture trace info if available
    trace_data = context_storage.get_current_context_instance().get_trace_data()
    if trace_data:
        main_context_provider.set_trace_id(trace_data.id)
        print(f"  Captured trace ID: {trace_data.id}")

    # Test 2: Specialist delegation
    print("\nTest 2: Specialist Delegation")
    response2 = main_agent.run(UserQuery(message="I need a specialist for this complex issue"))
    print(f"  Response: {response2.message}")
    print(f"  Context: {response2.trace_context}")

    print("\nObservation: Each agent.run() creates a separate trace (different IDs above)")
    print("This is the default behavior when using track_atomic_agents()")
    
    # Now demonstrate shared trace context concept
    print("\n" + "="*60)
    demonstrate_shared_trace_context()

    # Show context provider information
    print("\n" + "="*60)
    print("=== Context Provider Output Examples ===\n")
    
    # Update with a sample trace ID for demonstration
    sample_trace_id = "019828ba-sample-trace-id-demo"
    main_context_provider.set_trace_id(sample_trace_id)
    
    print("Main Agent Context Provider Output:")
    main_context = main_context_provider.get_info()
    if main_context:
        print(f"  {main_context}")
    
    # Show specialist context
    specialist_demo_provider = OpikContextProvider(
        project_name="context-provider-demo",
        trace_id=sample_trace_id,
        include_metadata=True,
        metadata={
            "agent_role": "specialist",
            "delegation_source": "main_agent",
            "processing_step": "analysis"
        }
    )
    
    print("\nSpecialist Agent Context Provider Output:")
    specialist_context = specialist_demo_provider.get_info()
    if specialist_context:
        print(f"  {specialist_context}")

    print("\n" + "="*60)
    print("=== Demo Complete ===")
    print("Key takeaways:")
    print("1. OpikContextProvider injects trace metadata into system prompts")
    print("2. Context providers can include custom metadata for better debugging")
    print("3. By default, each agent.run() creates a separate trace")
    print("4. For shared context, wrap workflows in opik_context.trace_context()")
    print("5. Shared context enables linking multi-agent conversations")
    print("6. Context providers help agents reference their execution context")
    print("7. trace_id can be set manually or detected automatically from context")


if __name__ == "__main__":
    main() 

=== OpikContextProvider Demonstration ===

1. Basic Context Provider:
   Without trace_id: ''
   With trace_id: '[Opik Trace Info]
trace_id: trace-12345
project:  demo-project'

2. Context Provider with Metadata:
   Full context:
[Opik Trace Info]
trace_id: trace-67890
project:  demo-project
agent_role: specialist
workflow_type: delegation
user_id: user-123

3. System Prompt Injection Example:
   When an agent uses this context provider, this information
   gets automatically injected into the system prompt:
   ```
   [Opik Trace Info]
trace_id: trace-67890
project:  demo-project
agent_role: specialist
workflow_type: delegation
user_id: user-123
   ```

=== Individual Agent Runs (Default Behavior) ===

Test 1: Regular Request
  Response: I'm the main agent. Ask for a 'specialist' to see delegation!
  Context: Main agent trace: 019828c2-9413-7da9-971f-f312dc402658

Test 2: Specialist Delegation
  Response: Specialist says: Specialist analysis complete! (Question: I need a specialist ...

OPIK: Failed to process CreateTraceBatchMessage. Error: headers: {'date': 'Sun, 20 Jul 2025 16:54:59 GMT', 'content-type': 'application/json', 'content-length': '51', 'connection': 'keep-alive', 'set-cookie': 'AWSALB=H2OqtqSbc05/Wlfv1Zl/MumFc4t1VWxFzj4tKEIaHN/IR5SnH3F/q2ihz8sLd0sn/tlrB/Zq6oFT0nN/tt0bi5H3Np6J8Qy86ZmxPs3qBpSHO9/TfeQXyPc5Yxkf; Expires=Sun, 27 Jul 2025 16:54:59 GMT; Path=/, AWSALBCORS=H2OqtqSbc05/Wlfv1Zl/MumFc4t1VWxFzj4tKEIaHN/IR5SnH3F/q2ihz8sLd0sn/tlrB/Zq6oFT0nN/tt0bi5H3Np6J8Qy86ZmxPs3qBpSHO9/TfeQXyPc5Yxkf; Expires=Sun, 27 Jul 2025 16:54:59 GMT; Path=/; SameSite=None; Secure', 'server': 'nginx'}, status_code: 401, body: {'code': 401, 'message': 'API key should be provided'}
